# Multilayer Perceptron (MLP)


Sources:

Sources:

- [3Blue1Brown video: But what is a neural network? | Deep learning chapter 1](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=1)
- [Stanford cs231n: Deep learning ](http://cs231n.stanford.edu/)
- [Pytorch: WWW tutorials](https://pytorch.org/tutorials/)
- [Pytorch: github tutorials](https://github.com/pytorch/tutorials)
- [Pytorch: github examples](https://github.com/pytorch/examples)
- [Pytorch examples](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
- [MNIST/pytorch nextjournal.com/gkoehler/pytorch-mnist](https://nextjournal.com/gkoehler/pytorch-mnist)
- [Pytorch: github/pytorch/examples](https://github.com/pytorch/examples/tree/master/mnist)
- [kaggle: MNIST/pytorch](https://www.kaggle.com/sdelecourt/cnn-with-pytorch-for-mnist)


In [ ]:
%matplotlib inline

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision import models
#
from pathlib import Path
# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Plot parameters
plt.style.use('seaborn-v0_8-whitegrid')
fig_w, fig_h = plt.rcParams.get('figure.figsize')
plt.rcParams['figure.figsize'] = (fig_w, fig_h * .5)
%matplotlib inline

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = 'cpu' # Force CPU
print(device)

## Single Layer Softmax Classifier (Multinomial Logistic Regression)

Recall of Binary logistic regression

<img src="figures/logistic.png" width="300">

One neuron as output layer
$$
f(\boldsymbol{x}) = \sigma(\boldsymbol{x}^{T} \boldsymbol{w} + b)
$$

Where

- Input: $\boldsymbol{x}$: a vector of dimension $(p)$ (layer 0).
- Parameters: $\boldsymbol{w}$: a vector of dimension $(p)$ (layer 1). $b$ is the scalar bias.
- Output: $f(\boldsymbol{x})$ a vector of dimension 1. 

With multinomial logistic regression we have $k$ possible labels to predict. If we consider the MNIST Handwritten Digit Recognition, the inputs is a $28 \times 28=784$ image and the output is a vector of $k=10$ labels or probabilities.

<img src="figures/logistic_multinominal.png" width="300">

$$
f(\boldsymbol{x}) = \text{softmax}(\boldsymbol{x}^{T} \boldsymbol{W} + \boldsymbol{b})
$$

- Input: $\boldsymbol{x}$: a vector of dimension $(p=784)$ (layer 0).
- Parameters: $\boldsymbol{W}$: the matrix of coefficients of dimension $(p \times k)$ (layer 1).
  $b$ is a $(k)$-dimentional vector of bias.
- Output: $f(\boldsymbol{x})$ a vector of dimension $(k=10)$ possible labels 

The softmax function is a crucial component in many machine learning and deep learning models, particularly in the context of classification tasks. It is used to convert a vector of raw scores (logits) into a probability distribution. Here's a detailed explanation of the softmax function:
The softmax function takes a vector of real numbers as input and outputs a vector of probabilities that sum to 1. The formula for the softmax function is:

$$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j} e^{z_j}}
$$

where:
- $z_i$ is the $i$-th element of the input vector $\mathbf{z}$.
- $e$ is the base of the natural logarithm.
- The sum in the denominator is over all elements of the input vector.

Softmax Properties

1. **Probability Distribution**: The output of the softmax function is a probability distribution, meaning that all the outputs are non-negative and sum to 1.
2. **Exponential Function**: The use of the exponential function ensures that the outputs are positive and that larger input values correspond to larger probabilities.
3. **Normalization**: The softmax function normalizes the input values by dividing by the sum of the exponentials of all input values, ensuring that the outputs sum to 1



MNIST classfification using multinomial logistic

<img src="figures/logistic_multinominal_MNIST.png" width="800">

[source: Logistic regression MNIST](https://notebooks.azure.com/cntk/projects/edxdle/html/Lab2_LogisticRegression.ipynb)

Here we fit a multinomial logistic regression with L2 penalty on a subset of
the MNIST digits classification task.

[source: scikit-learn.org](https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html)

Hyperparameters

## Dataset: MNIST Handwritten Digit Recognition

[MNIST Loader](https://github.com/duchesnay/pystatsml/blob/master/lib/pystatsml/datasets.py)

In [ ]:
from pystatsml.datasets import load_mnist_pytorch

dataloaders, WD = load_mnist_pytorch(batch_size_train=64, batch_size_test=10000)
os.makedirs(os.path.join(WD, "models"), exist_ok=True)

# Info about the dataset
D_in = np.prod(dataloaders["train"].dataset.data.shape[1:])
D_out = len(dataloaders["train"].dataset.targets.unique())
print("Datasets shapes:", {x: dataloaders[x].dataset.data.shape for x in ['train', 'test']})
print("N input features:", D_in, "Output classes:", D_out)

Now let's take a look at some mini-batches examples.


In [ ]:
batch_idx, (example_data, example_targets) = next(enumerate(dataloaders["train"]))
print("Train batch:", example_data.shape, example_targets.shape)
batch_idx, (example_data, example_targets) = next(enumerate(dataloaders["test"]))
print("Val batch:", example_data.shape, example_targets.shape)

So one test data batch is a tensor of shape: . This means we have 1000 examples of 28x28 pixels in grayscale
(i.e. no rgb channels, hence the one). We can plot some of them using matplotlib.



In [ ]:
def show_data_label_prediction(data, y_true, y_pred=None, shape=(2, 3)):
    y_pred = [None] * len(y_true) if y_pred is None else y_pred
    fig = plt.figure()
    for i in range(np.prod(shape)):
        plt.subplot(*shape, i+1)
        plt.tight_layout()
        plt.imshow(data[i][0], cmap='gray', interpolation='none')
        plt.title("True: {} Pred: {}".format(y_true[i], y_pred[i]))
        plt.xticks([])
        plt.yticks([])

show_data_label_prediction(data=example_data, y_true=example_targets, y_pred=None, shape=(2, 3))

In [ ]:
X_train = dataloaders["train"].dataset.data.numpy()
X_train = X_train.reshape((X_train.shape[0], -1))
y_train = dataloaders["train"].dataset.targets.numpy()

X_test = dataloaders["test"].dataset.data.numpy()
X_test = X_test.reshape((X_test.shape[0], -1))
y_test = dataloaders["test"].dataset.targets.numpy()

print(X_train.shape, y_train.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Turn up tolerance for faster convergence
#clf = LogisticRegression(C=50., multi_class='multinomial', solver='sag', tol=0.1)
clf = LogisticRegression(C=50., solver='sag', tol=0.1)
clf.fit(X_train, y_train)
#sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)

print("Test score with penalty: %.4f" % score)

In [ ]:
coef = clf.coef_.copy()
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(10):
    l1_plot = plt.subplot(2, 5, i + 1)
    l1_plot.imshow(coef[i].reshape(28, 28), interpolation='nearest',
                   cmap=plt.cm.RdBu, vmin=-scale, vmax=scale)
    l1_plot.set_xticks(())
    l1_plot.set_yticks(())
    l1_plot.set_xlabel('Class %i' % i)
plt.suptitle('Classification vector for...')

plt.show()

## Model: Two Layer MLP

### MLP with Scikit-learn

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=10, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=0.01, batch_size=64)

mlp.fit(X_train, y_train)
print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

print("Coef shape=", len(mlp.coefs_))

fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=.5 * vmin,
               vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

plt.show()

### MLP with pytorch

In [ ]:
class TwoLayerMLP(nn.Module):

    def __init__(self, d_in, d_hidden, d_out):
        super(TwoLayerMLP, self).__init__()
        self.d_in = d_in
        
        self.linear1 = nn.Linear(d_in, d_hidden)
        self.linear2 = nn.Linear(d_hidden, d_out)

    def forward(self, X):
        X = X.view(-1, self.d_in)
        X = self.linear1(X)
        return F.log_softmax(self.linear2(X), dim=1)

#### Train the Model

- First we want to make sure our network is in training mode.

- Iterate over epochs

- Alternate train and validation dataset

- Iterate over all training/val data once per epoch. Loading the individual batches is handled by the DataLoader.

- Set the gradients to zero using `optimizer.zero_grad()` since PyTorch by default accumulates gradients.

- Forward pass:
  * `model(inputs)`: Produce the output of our network.
  * `torch.max(outputs, 1)`: softmax predictions.
  * `criterion(outputs, labels)`: loss between the output and the ground truth label.
                    
- In training mode, backward pass `backward()`: collect a new set of gradients which we propagate back into each of the network's parameters using `optimizer.step()`.

- We'll also keep track of the progress with some printouts. In order to create a nice training curve later on we also create two lists for saving training and testing losses. On the x-axis we want to display the number of training examples the network has seen during training.

- Save model state: Neural network modules as well as optimizers have the ability to save and load their internal state using `.state_dict()`. With this we can continue training from previously saved state dicts if needed - we'd just need to call `.load_state_dict(state_dict)`.

In [ ]:
# %load train_val_model.py
from pystatsml.dl_utils import train_val_model

Run one epoch and save the model

In [ ]:
model = TwoLayerMLP(D_in, 50, D_out).to(device)
print(next(model.parameters()).is_cuda)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.NLLLoss()

# Explore the model
for parameter in model.parameters():
    print(parameter.shape)

print("Total number of parameters =", np.sum([np.prod(parameter.shape) for parameter in model.parameters()]))

model, losses, accuracies = train_val_model(model, criterion, optimizer, dataloaders,
                       num_epochs=1, log_interval=1)

print(next(model.parameters()).is_cuda)
torch.save(model.state_dict(), 
           os.path.join(WD, 'models/mod-%s.pth' % model.__class__.__name__))

Use the model to make new predictions. Consider the device, ie, load data on device `example_data.to(device)` from prediction, then move back to cpu `example_data.cpu()`.

In [ ]:
batch_idx, (example_data, example_targets) = next(enumerate(dataloaders["test"]))
example_data = example_data.to(device)

with torch.no_grad():
  output = model(example_data).cpu()

example_data = example_data.cpu()

# Softmax predictions 
preds = output.argmax(dim=1)

print("Output shape=", output.shape, "label shape=", preds.shape)
print("Accuracy = {:.2f}%".format((example_targets == preds).sum().item() * 100. / len(example_targets)))

show_data_label_prediction(data=example_data, y_true=example_targets, y_pred=preds, shape=(3, 4))

Plot missclassified samples

In [ ]:
errors = example_targets != preds
#print(errors, np.where(errors))
print("Nb errors = {}, (Error rate = {:.2f}%)".format(errors.sum(), 100 * errors.sum().item() / len(errors)))
err_idx = np.where(errors)[0]
show_data_label_prediction(data=example_data[err_idx], y_true=example_targets[err_idx],
                           y_pred=preds[err_idx], shape=(3, 4))

#### Continue training from checkpoints: reload the model and run 10 more epochs

In [ ]:
model = TwoLayerMLP(D_in, 50, D_out)
model.load_state_dict(torch.load(os.path.join(WD, 'models/mod-%s.pth' % model.__class__.__name__)))
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.NLLLoss()

model, losses, accuracies = train_val_model(model, criterion, optimizer, dataloaders,
                       num_epochs=10, log_interval=2)

_ = plt.plot(losses['train'], '-b', losses['test'], '--r')

## Test several MLP architectures

- Define a `MultiLayerMLP([D_in, 512, 256, 128, 64, D_out])` class that take the size of the layers as parameters of the constructor.
- Add some non-linearity with relu acivation function

In [ ]:
class MLP(nn.Module):

    def __init__(self, d_layer):
        super(MLP, self).__init__()
        self.d_layer = d_layer
        # Add linear layers
        layer_list = [nn.Linear(d_layer[l], d_layer[l+1])
                      for l in range(len(d_layer) - 1)]
        self.linears = nn.ModuleList(layer_list)

    def forward(self, X):
        X = X.view(-1, self.d_layer[0])
        # relu(Wl x) for all hidden layer
        for layer in self.linears[:-1]:
            X = F.relu(layer(X))
        # softmax(Wl x) for output layer
        return F.log_softmax(self.linears[-1](X), dim=1)

In [ ]:
model = MLP([D_in, 512, 256, 128, 64, D_out]).to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.NLLLoss()

model, losses, accuracies = \
    train_val_model(model, criterion, optimizer, dataloaders,
                    num_epochs=10, log_interval=2)

_ = plt.plot(losses['train'], '-b', losses['test'], '--r')

## Reduce the size of training dataset

Reduce the size of the training dataset by considering only `10` minibatche for size`16`.

In [ ]:
train_size = 10 * 16

# Stratified sub-sampling
targets = dataloaders["train"].dataset.targets.numpy()
nclasses = len(set(targets))

indices = np.concatenate([np.random.choice(np.where(targets == lab)[0],
                                           int(train_size / nclasses),replace=False) 
    for lab in set(targets)])
np.random.shuffle(indices)

train_loader = torch.utils.data.DataLoader(dataloaders["train"].dataset,
                                           batch_size=16,
        sampler=torch.utils.data.SubsetRandomSampler(indices))

# Check train subsampling
train_labels = np.concatenate([labels.numpy() for inputs, labels in train_loader])
print("Train size=", len(train_labels), " Train label count=",
      {lab:np.sum(train_labels == lab) for lab in set(train_labels)})
print("Batch sizes=", [inputs.size(0) for inputs, labels in train_loader])

# Put together train and val
dataloaders = dict(train=train_loader, test=dataloaders["test"])
                   
# Info about the dataset
D_in = np.prod(dataloaders["train"].dataset.data.shape[1:])
D_out = len(dataloaders["train"].dataset.targets.unique())
print("Datasets shape", {x: dataloaders[x].dataset.data.shape
                         for x in dataloaders.keys()})
print("N input features", D_in, "N output", D_out)

In [ ]:
model = MLP([D_in, 512, 256, 128, 64, D_out]).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.NLLLoss()

model, losses, accuracies = \
    train_val_model(model, criterion, optimizer, dataloaders,
                    num_epochs=100, log_interval=20)

_ = plt.plot(losses['train'], '-b', losses['test'], '--r')

Use an opimizer with an adaptative learning rate: Adam

In [ ]:
model = MLP([D_in, 512, 256, 128, 64, D_out]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

model, losses, accuracies = train_val_model(model, criterion, optimizer, dataloaders,
                       num_epochs=100, log_interval=20)

_ = plt.plot(losses['train'], '-b', losses['test'], '--r')

## Run MLP on CIFAR-10 dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

Here are the classes in the dataset, as well as 10 random images from each:
- airplane 										
- automobile 										
- bird 										
- cat 										
- deer 										
- dog 										
- frog 										
- horse 										
- ship 										
- truck

Load CIFAR-10 dataset [CIFAR-10 Loader](https://github.com/duchesnay/pystatsml/blob/master/lib/pystatsml/datasets.py)

In [ ]:
from pystatsml.datasets import load_cifar10_pytorch

dataloaders, _ = load_cifar10_pytorch(batch_size_train=100, batch_size_test=100)

# Info about the dataset
D_in = np.prod(dataloaders["train"].dataset.data.shape[1:])
D_out = len(set(dataloaders["train"].dataset.targets))
print("Datasets shape:", {x: dataloaders[x].dataset.data.shape for x in dataloaders.keys()})
print("N input features:", D_in, "N output:", D_out)

Run MLP Classifier with hidden layers of sizes: 512, 256, 128, and 64:

In [ ]:
model = MLP([D_in, 512, 256, 128, 64, D_out]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

model, losses, accuracies = \
    train_val_model(model, criterion, optimizer, dataloaders,
                       num_epochs=50, log_interval=10)

_ = plt.plot(losses['train'], '-b', losses['test'], '--r')